In [33]:
import ccxt, time, csv, os, string

In [52]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    for _ in range(max_retries):    
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
            if ohlcv is not None:
                return ohlcv
        except Exception:
            continue
    raise Exception('Failed to fetch {} in {} attempts'.format(symbol, max_retries))

def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
#         print('{} candles from {} to {}'.format(
#             len(all_ohlcv),
#             exchange.iso8601(all_ohlcv[0][0]),
#             exchange.iso8601(all_ohlcv[-1][0])
#         ))
        if fetch_since < since:
            break
    return all_ohlcv


def write_csv(filename, data):
    with open(filename, 'w+') as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['timestamp', 'o', 'h', 'l', 'c', 'v'])
        writer.writerows(data)


def scrape_candles_to_csv(exchange, filename, max_retries, symbol, timeframe, since, limit):
    # convert start time from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    write_csv(filename, ohlcv)
    print('Wrote {} candles from {} to {} in {}'.format(
        len(ohlcv),
        exchange.iso8601(ohlcv[0][0]),
        exchange.iso8601(ohlcv[-1][0]),
        filename
    ))

In [ ]:
exchanges = ['binance', 'bittrex', 'bitmex', 'coss']
pairs = ['BTC/USDT', 'ETH/USDT', 'BTC/USD', 'ETH/USD']
start = '2019-03-01T00:00:00Z'
tick = '1m'
ticks_per_batch = 12 * 60
max_attempts = 5

for exchange_id in exchanges:
    exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})
    exchange.load_markets()
    if not exchange.has['fetchOHLCV']:
        print('{} does not expose OHLC data'.format(ex.id))
        continue
    os.makedirs('data/fine/{}'.format(exchange_id), exist_ok=True)
    for pair in pairs:
        if not pair in exchange.symbols:
            # print('Exchange {} does not trade {}'.format(exchange_id, pair))
            continue
        print('Downloading price history for {} on exchange {}'.format(pair, exchange_id))
        filename = 'data/fine/{}/{}.csv'.format(exchange_id, pair.replace('/', '-'))
        scrape_candles_to_csv(exchange, filename, max_attempts, pair, tick, start, ticks_per_batch)

Wrote 17280 candles from 2019-02-28T21:47:00.000Z to 2019-03-12T21:46:00.000Z in data/fine/binance/BTC-USDT.csv
Wrote 17280 candles from 2019-02-28T21:48:00.000Z to 2019-03-12T21:47:00.000Z in data/fine/binance/ETH-USDT.csv
Wrote 16976 candles from 2019-02-28T22:46:00.000Z to 2019-03-12T21:43:00.000Z in data/fine/bittrex/BTC-USDT.csv
Wrote 16950 candles from 2019-02-28T12:27:00.000Z to 2019-03-12T21:35:00.000Z in data/fine/bittrex/ETH-USDT.csv
Wrote 16918 candles from 2019-02-28T22:14:00.000Z to 2019-03-12T21:46:00.000Z in data/fine/bittrex/BTC-USD.csv
Wrote 15421 candles from 2019-02-28T15:02:00.000Z to 2019-03-12T21:47:00.000Z in data/fine/bittrex/ETH-USD.csv
